In [9]:
import pandas as pd
import numpy as np

#### Data 1: lei(2018)_tibet_lakes

In [ ]:
path_1 = 'data/ground-observation/lei(2018)_tibet_lakes/tibet_lakes.xlsx'
df_1 = pd.read_excel(path_1, usecols=['sno', 'date', 'waterLevel'])
df_1 = df_1.rename(columns={'sno': 'lake_name','waterLevel':'water_level'})
print(df_1.head())

d:\miniconda3\envs\courses\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


    lake_name        date  water_level
0  zhariNamco  2010-04-01        0.323
1  zhariNamco  2010-04-02        0.322
2  zhariNamco  2010-04-03        0.320
3  zhariNamco  2010-04-04        0.328
4  zhariNamco  2010-04-05        0.310


#### Data 2: lei(2022)_tibet_lakes

In [ ]:
lake_columns_config = {
    'Lumajiangdong_Co': {'date_col': 'date', 'water_level_col': 'LMJD_water_level (m)'},
    'Memar_Co': {'date_col': 'date', 'water_level_col': 'Memar Co_water_level (m)'},
    'Luotuo_lake': {'date_col': 'date', 'water_level_col': 'Luotuo lake_water_level'},
    'Jieze_Caka': {'date_col': 'date', 'water_level_col': 'Jieze Caka_water level'}
    }

def read_lakes_safely(file_path, config):
    
    df = pd.read_excel(file_path)
    lakes_data = {}
    for lake_name, cols in config.items():
        if all(col in df.columns for col in cols.values()):
            lake_df = df[list(cols.values())].dropna().rename(columns=dict(zip(cols.values(), ['date', 'water_level'])))
            lake_df['lake_name'] = lake_name
            lakes_data[lake_name] = lake_df
            print(f"Successfully read {lake_name}: {lake_df.shape[0]} 行")
        else:
            print(f"{lake_name} does not exist")
    
    return lakes_data

path_2 = 'data/ground-observation/lei(2022)_tibet_lakes/lei(2022)_tibet_lakes.xlsx'
df_2_dict = read_lakes_safely(path_2, lake_columns_config)
df_2 = pd.concat(df_2_dict.values(), ignore_index=True)

print("Total Data:")
print(f"total {len(df_2)} rows of data")
print(df_2.head()) 

Successfully read Lumajiangdong_Co: 1451 行
Successfully read Memar_Co: 694 行
Successfully read Luotuo_lake: 259 行
Successfully read Jieze_Caka: 332 行
Total Data:
total 2736 rows of data
        date  water_level         lake_name
0 2016-09-30     0.875692  Lumajiangdong_Co
1 2016-10-01     0.876787  Lumajiangdong_Co
2 2016-10-02     0.873771  Lumajiangdong_Co
3 2016-10-03     0.873217  Lumajiangdong_Co
4 2016-10-04     0.866892  Lumajiangdong_Co


#### Data 3: wang(2018)_namco

In [ ]:
path_3 = 'data/ground-observation/wang(2018)_namco/wang(2018)_namco.xls'
df_3 = pd.read_excel(path_3, usecols=['date', 'water_level'])
df_3['lake_name'] = 'Namco'
df_3['water_level'] = df_3['water_level'] / 100
print(df_3.head())

        date  water_level lake_name
0 2007-01-02          NaN     Namco
1 2007-01-03          NaN     Namco
2 2007-01-04          NaN     Namco
3 2007-01-05          NaN     Namco
4 2007-01-06          NaN     Namco


#### Data 4: xie(2021)_kalakuli_lake

In [ ]:
path_4 = 'data/ground-observation/xie(2021)_kalakuli_lake/卡拉库里湖水位（2011-2019）.xlsx'
column_config = {year: {'date': '时间', 'water': '水位（cm）'} for year in map(str, range(2011, 2020))}

result_data = []
for year, cols in column_config.items():
    try:
        df = pd.read_excel(path_4, sheet_name=year)
        temp_df = df[list(cols.values())].copy()
        temp_df.columns = ['date', 'water_level']
        temp_df['lake_name'] = 'kalakuli'
        temp_df['water_level'] /= 100  # cm转m
        result_data.append(temp_df)
    except Exception as e:
        print(f"{year} year failed: {e}")

df_4 = pd.concat(result_data, ignore_index=True) if result_data else pd.DataFrame()
print(f"Integration complete! A total of {len(df_4)} data points")
print(df_4.head()) 

Integration complete! A total of 834 data points
        date  water_level lake_name
0 2011-08-23     0.000000  kalakuli
1 2011-08-24     0.038735  kalakuli
2 2011-08-25     0.073385  kalakuli
3 2011-08-26     0.092691  kalakuli
4 2011-08-27     0.101668  kalakuli


#### Data 5: zhang2018_silingco

In [ ]:
def read_lake_data(file_path):
    """read data"""
    df = pd.read_excel(file_path, sheet_name='气象数据集', skiprows=2)
    df = df[df.columns[1:4]].copy()
    df.columns = ['时间', '压强', '水温']
    df['时间'] = df['时间'].apply(lambda x: pd.to_datetime(
        str(x).strip().replace('上午', 'AM').replace('下午', 'PM'), 
        format='%m/%d/%y %p%I时%M分%S秒', errors='coerce'))  # type: ignore
    return df.dropna(subset=['时间', '压强', '水温'])

def calculate_instant_water_depth(df, altitude=4551, salinity=0):
    """Calculate the instantaneous water depth for all data points"""
    P_atm = 101325 * (1 - 0.0065 * altitude / 288.15) ** (9.80665 / (287.05 * 0.0065))    

    df['瞬时水深'] = df.apply(lambda row: 
        (row['压强'] * 1000 - P_atm) / 
        ((1000 * (1 - (row['水温'] + 288.9414) * (row['水温'] - 3.9863)**2 / 
        (508929.2 * (row['水温'] + 68.12963))) + 0.8 * salinity) * 9.81), axis=1)
    
    df['日期'] = df['时间'].dt.date
    df['时间'] = df['时间'].dt.time
    df['lake_name'] = 'silingco'
    
    result_df = df[['日期', '时间', '瞬时水深', 'lake_name']].copy()

    result_df = result_df.rename(columns={
        '日期': 'date', 
        '时间': 'time', 
        '瞬时水深': 'water_level'
    })
    
    return result_df

def create_complete_time_series(df_result):
    """Create a complete time series with other time columns set to NaN"""
    min_date = df_result['date'].min()
    max_date = df_result['date'].max()
    all_dates = pd.date_range(start=min_date, end=max_date, freq='D').date
 
    complete_df = pd.DataFrame({
        'date': all_dates,
        'lake_name': 'silingco'
    })
    
    merged_df = pd.merge(complete_df, df_result, on=['date', 'lake_name'], how='left')
    merged_df = merged_df[['date', 'time', 'water_level', 'lake_name']]
    
    return merged_df

file_path = "data/ground-observation/zhang(2018)_silingco/zhang(2018)_silingco.xls"
df = read_lake_data(file_path)

print(f"Data Time Period: {df['时间'].min()} 到 {df['时间'].max()}")
print(f"Total number of data points: {len(df)}")

df_result = calculate_instant_water_depth(df, altitude=4551)

df_5 = create_complete_time_series(df_result)

print("完整时间序列的水位计算结果：")
print(df_5)


Data Time Period: 2016-09-17 08:00:00 到 2017-02-15 15:00:00
Total number of data points: 3632
完整时间序列的水位计算结果：
            date      time  water_level lake_name
0     2016-09-17  08:00:00     0.160833  silingco
1     2016-09-17  09:00:00     0.167652  silingco
2     2016-09-17  10:00:00     0.153089  silingco
3     2016-09-17  11:00:00     0.210122  silingco
4     2016-09-17  12:00:00     0.414559  silingco
...          ...       ...          ...       ...
3627  2017-02-15  11:00:00     1.209611  silingco
3628  2017-02-15  12:00:00     0.970514  silingco
3629  2017-02-15  13:00:00     0.677170  silingco
3630  2017-02-15  14:00:00     0.640451  silingco
3631  2017-02-15  15:00:00     0.611175  silingco

[3632 rows x 4 columns]


#### Statistics of all in-situ data

In [ ]:
dfs = [df_1, df_2, df_3, df_4, df_5]
for i in range(4):  # Add a time column to the first 4 DataFrames
    dfs[i]['time'] = np.nan

df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[['date', 'water_level', 'lake_name', 'time']]
df_all['date'] = pd.to_datetime(df_all['date']).dt.strftime('%Y-%m-%d')
print(df_all['lake_name'].value_counts())

##output
output_path = "data/ground-observation/hydro_station_lakes_all.csv"
df_all.to_csv(output_path, index=False)
print(f"\nData saved to: {output_path}")

lake_name
Namco               3652
silingco            3632
zhariNamco          2616
BamCo               1461
Lumajiangdong_Co    1451
PengCo              1257
DazegCo             1104
DawaCo               867
kalakuli             834
Memar_Co             694
Jieze_Caka           332
Luotuo_lake          259
Name: count, dtype: int64

Data saved to: D:\Desktop\insitu_code\hydro_station_lakes_all.csv
